In [53]:
#Импорты
import pandas as pd
import re 

In [54]:
#Читаем датасет
rating_movies = pd.read_csv('data/movies_data/ratings_movies.csv')
display(rating_movies)

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi


In [55]:

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [56]:
#1
rating_movies['year_release'] = rating_movies['title'].apply(get_year_release)
rating_movies['year_release'] = rating_movies['year_release'].astype('Int64')
rating_movies['date'] = pd.to_datetime(rating_movies['date'])
display(rating_movies.info())
display(rating_movies)
#2
worstFilmsOf1999Mask = rating_movies[rating_movies['year_release'] == 1999].sort_values(by='title')
worstFilmsOf1999 = worstFilmsOf1999Mask.groupby('title')['rating'].mean().sort_values()
display('Фильм с худшим рейтинком в 1999 году:',
      worstFilmsOf1999.nsmallest(1)) #Bloodsport: The Dark Kumite
#3
worstGenresOf2010Mask = rating_movies[rating_movies['year_release'] == 2010].sort_values(by='title')
worstGenresOf2010 = worstGenresOf2010Mask.groupby(['genres'])['rating'].mean().sort_values()
display('Жанры с худшим рейтигом в 2010 году:',
      worstGenresOf2010.nsmallest(1)) #Action|Sci-Fi
#4
display('ID пользователя, посмотревшего наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов', \
    rating_movies.groupby('userId')['genres'].nunique().nlargest(1)) #599
#5
display('ID пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая:', 
      rating_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], \
          ascending=[True, False]).head(1)) #53
#6
groupingByUserID = rating_movies.groupby('userId')['rating'].agg(['count', 'mean']) #Вспомогательное
groupingByGenres = rating_movies.groupby('genres')['rating'].agg(['count', 'mean']) #Вспомогательное, не используется
genresOf2018 = rating_movies[rating_movies['year_release'] == 2018]
genresWith10OrMoreMarks = genresOf2018.groupby('genres')['rating'].agg(['count', 'mean'])
genresWith10OrMoreMarks = genresWith10OrMoreMarks[genresWith10OrMoreMarks['count'] > 10]
display('Cочетание жанров за 2018 год, которое имеет наибольший средний рейтинг,', 
        'и при этом число выставленных ему оценок больше 10:', genresWith10OrMoreMarks) #Action|Adventure|Sci-Fi
#7
rating_movies['year_rating'] = rating_movies['date'].dt.year
mask1 = rating_movies[(rating_movies['genres'] == 'Action|Adventure')]
display(mask1.groupby('year_rating')['rating'].agg('mean')) #True
mask2 = rating_movies[(rating_movies['genres'] == 'Action|Adventure|Animation|Children|Comedy|IMAX)') & (rating_movies['year_rating'] == 2010)]
display(mask2) #False
mask3 = rating_movies[(rating_movies['year_rating'] == 2018) & (rating_movies['genres'] == 'Animation|Children|Mystery')]
display(mask3) #True
mask4 = rating_movies[rating_movies['genres'] == 'Comedy']
display(mask4.groupby('year_rating')['rating'].agg('mean')) #False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  Int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 6.3+ MB


None

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995
...,...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017


'Фильм с худшим рейтинком в 1999 году:'

title
Bloodsport: The Dark Kumite (1999)    0.5
Name: rating, dtype: float64

...

,count,mean
genres,,
Action|Adventure|Sci-Fi,14,3.928571
Action|Comedy|Sci-Fi,12,3.875000


year_rating
1996    3.454545
1997    4.150000
1998    4.200000
1999    4.000000
2000    3.738462
2001    3.500000
2002    4.304348
2003    3.277778
2004    4.136364
2005    3.413043
2006    4.090909
2007    3.666667
2008    3.656250
2009    3.714286
2010    3.437500
2011    3.888889
2012    3.900000
2013    3.833333
2014    3.875000
2015    3.560976
2016    3.722222
2017    3.527778
2018    3.397436
Name: rating, dtype: float64

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release,year_rating


,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release,year_rating
13930,13930,89,118894,5.0,2018-03-07 07:51:01,Scooby-Doo! Abracadabra-Doo (2010),Animation|Children|Mystery,2010,2018


year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: rating, dtype: float64

А теперь заказы в интернет-магазине

In [60]:
orders = pd.read_csv('data/orders.csv', sep=';')
display(orders)
products = pd.read_csv('data/products.csv', sep=';')
display(products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [78]:
# Создаём общую таблицу

products = products.rename(columns={'Product_ID':'ID товара'})
orders_products = orders.merge(
    products, on='ID товара', how='left')

print('Название отменённого товара:', orders_products[orders_products['Отменен'] == 'Да']['Name'])
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество']
display(orders_products)
display(orders_products[orders_products['Оплачен'] == 'Да'] \
    .groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False))

Название отменённого товара: 6    Носки беговые Camino
Name: Name, dtype: object


,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Name,Price,CURRENCY,Profit
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR,995.0
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,"Носки Простые, муж",45.0,RUR,4500.0
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,"Носки Подарочные, жен",249.0,RUR,2490.0
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,"Носки Подарочные, жен",249.0,RUR,1743.0
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,"Носки Подарочные, жен",249.0,RUR,1245.0
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR,995.0
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,Носки беговые Camino,999.0,RUR,999.0
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,"Носки Честные, муж",50.0,RUR,50.0
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,"Носки Подарочные, муж",199.0,RUR,597.0
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,"Носки Подарочные, жен",249.0,RUR,747.0


ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Profit, dtype: float64